### curriculum - 【AIコース】強化学習　入門編3 Policy gradient methodとBandit問題

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [2]:
#C Facing error: 
#C AttributeError: module 'pandas' has no attribute 'computation'
#C fixed by running...
#C pip install dask --upgrade
#C Thanks to...
#C https://github.com/tflearn/tflearn/issues/766

In [3]:
bandit_arms = [0.2, 0, -0.2, -2]
num_arms = len(bandit_arms)

def pullBandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

In [4]:
tf.reset_default_graph()
weights = tf.Variable(tf.ones([num_arms]))
output = tf.nn.softmax(weights)

In [7]:
reward_holder = tf.placeholder(shape = [1], dtype = tf.float32)
action_holder = tf.placeholder(shape = [1], dtype = tf.int32)

responsible_output = tf.slice(output, action_holder, [1])
loss = -(tf.log(responsible_output) * reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-3)
update = optimizer.minimize(loss)

In [11]:
total_episodes = 1000
total_reward = np.zeros(num_arms)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        actions = sess.run(output)
        a = np.random.choice(actions, p = actions)
        action = np.argmax(actions == a)
        reward = pullBandit(bandit_arms[action])
        _, resp, ww = sess.run([update, responsible_output, weights], 
                               feed_dict = {reward_holder: [reward],
                                            action_holder: [action]})
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the" + str(num_arms) + \
                  "arms of the bandit: " + str(total_reward))
        i += 1
print("\nThe most promising arm is "+ str(np.argmax(ww) + 1))

if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("The estimation is correct")
else:
    print("The estimation is wrong")

Running reward for the4arms of the bandit: [1. 0. 0. 0.]
Running reward for the4arms of the bandit: [-5.  3.  6. 13.]
Running reward for the4arms of the bandit: [-1.  4. 11. 21.]
Running reward for the4arms of the bandit: [-7.  2. 15. 33.]
Running reward for the4arms of the bandit: [-4.  1. 19. 49.]
Running reward for the4arms of the bandit: [-12.  -2.  20.  57.]
Running reward for the4arms of the bandit: [-9. -2. 25. 73.]
Running reward for the4arms of the bandit: [-11.  -1.  22.  87.]
Running reward for the4arms of the bandit: [-19.   4.  24. 102.]
Running reward for the4arms of the bandit: [-28.   4.  24. 123.]
Running reward for the4arms of the bandit: [-30.   2.  27. 136.]
Running reward for the4arms of the bandit: [-28.   1.  30. 156.]
Running reward for the4arms of the bandit: [-31.   4.  30. 172.]
Running reward for the4arms of the bandit: [-29.   8.  31. 185.]
Running reward for the4arms of the bandit: [-20.   5.  28. 202.]
Running reward for the4arms of the bandit: [-22.   5.